If the process crashes while we are waiting for Anthropic batch jobs to complete, this code can pull the results down when they are ready and put them in the cache

In [4]:
from repsheet_backend.genai import anthropic, genai_cache
import asyncio

In [7]:
SAVED_BATCH_IDS_FILE = "repsheet_backend/data/saved_batch_ids.txt"

with open(SAVED_BATCH_IDS_FILE, "r") as f:
    saved_batch_ids = f.read().splitlines()

async def save_batch_to_cache(batch_id: str):
    if batch_id in saved_batch_ids:
        print(f"Batch {batch_id} already saved")
        return
    results = await asyncio.to_thread(anthropic.messages.batches.results, batch_id)
    saved_count = 0
    cache_set_jobs = []
    for result in results:
        cache_key = result.custom_id
        # there are some old jobs with numeric custom_ids
        if len(cache_key) > 10:
            cache_set_jobs.append(genai_cache.set(cache_key, result.result.message.content[0].text))
            saved_count += 1
    await asyncio.gather(*cache_set_jobs)
    print(f"Saved {saved_count} results for {batch_id}")
    saved_batch_ids.append(batch_id)

In [8]:
await asyncio.gather(*[
    save_batch_to_cache(message.id)
    for message in anthropic.messages.batches.list()
    if message.processing_status == "ended"
])

for message in anthropic.messages.batches.list():
    if message.processing_status != "ended":
        print(f"Batch {message.id} is still {message.processing_status}")



Saved 0 results for msgbatch_012arDNFnmWQ77p1E2WLu7JM
Saved 0 results for msgbatch_01EciNEfCLG57CSof2mhJhSg
Saved 1 results for msgbatch_012E6aERzmoStuit7nghobCC
Saved 1 results for msgbatch_015JruEfXYS443wjtvR44Tfu
Saved 1 results for msgbatch_01BVGT8PrSECWc5mEuDMcm2q
Saved 1 results for msgbatch_011UWoaFT4d32ykixGqaKBho
Saved 1 results for msgbatch_019U74kaM9jkwsjtjmfc6HBD
Saved 1 results for msgbatch_01HnLJpT89RGQqvNDUWGDs1x
Saved 1 results for msgbatch_014aMUr8Cv46fM7wbrNh6mso
Saved 1 results for msgbatch_01N51foigwrjtusn7dEmExXj
Saved 1 results for msgbatch_01WK2zAYu47yXdB2H2cvD4AG
Saved 1 results for msgbatch_016xmH41PvqfmjSiQNbkbvoK
Saved 1 results for msgbatch_01RJBNEtrDFvVuC2XAvEWjhK
Saved 1 results for msgbatch_01EVrVnt6urEGgTtodkWVy2j
Saved 1 results for msgbatch_01VMHWkquLdTJPZFDGWtNcRi
Saved 1 results for msgbatch_0129eqQX4ytYrPnqVcnnsWwa
Saved 1 results for msgbatch_01HStsvDn1CsD5VWEZVBbrct
Saved 1 results for msgbatch_016W8Wi8NibpqcbXA19nU8xm
Saved 1 results for msgbatch

In [9]:
with open(SAVED_BATCH_IDS_FILE, "w") as f:
    f.write("\n".join(saved_batch_ids))